<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Train Practice

## *Data Science Unit 4 Sprint 2 Assignment 3*

Continue to use TensorFlow Keras & a sample of the [Quickdraw dataset](https://github.com/googlecreativelab/quickdraw-dataset) to build a sketch classification model. The dataset has been sampled to only 10 classes and 10000 observations per class. Using your baseline model from yesterday, hyperparameter tune it and report on your highest validation accuracy. Your singular goal today is to achieve the highest accuracy possible.

*Don't forgot to switch to GPU on Colab!*

### Hyperparameters to Tune

At a minimum, tune each of these hyperparameters using any strategy we discussed during lecture today: 
- Optimizer
- Learning Rate
- Activiation Function
  - At least 1 subparameter within the Relu activation function
- Number of Neurons in Hidden Layers
- Number of Hidden Layers
- Weight Initialization

In [1]:
%load_ext tensorboard

In [2]:
import tensorflow as tf
import numpy as np
import os

URL_ = "https://github.com/LambdaSchool/DS-Unit-4-Sprint-2-Neural-Networks/blob/main/quickdraw10.npz?raw=true"

path_to_zip = tf.keras.utils.get_file('./quickdraw10.npz', origin=URL_, extract=False)

In [3]:
data = np.load(path_to_zip)

X = data['arr_0']
y = data['arr_1']

print(X.shape)
print(y.shape)

(100000, 784)
(100000,)


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((67000, 784), (33000, 784), (67000,), (33000,))

In [5]:
class_names = ['apple',
 'anvil',
 'airplane',
 'banana',
 'The Eiffel Tower',
 'The Mona Lisa',
 'The Great Wall of China',
 'alarm clock',
 'ant',
 'asparagus']

In [7]:
X_train.shape, y_train.shape

((67000, 784), (67000,))

In [8]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

# this is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_img = Input(shape=(784,))

# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_img)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(784, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

In [9]:
# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

In [10]:
autoencoder.compile(optimizer='nadam', loss='binary_crossentropy')

In [11]:
X_train = X_train / 255.0
print(X_train.shape)

(67000, 784)


In [12]:
import os
import datetime

from tensorflow.keras.callbacks import TensorBoard
# tf.keras.callbacks.TesnorBoard()

stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=2)
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

logdir = os.path.join("logs", f"SimpleAutoencoder-{now}")
tensorboard = TensorBoard(log_dir=logdir)

autoencoder.fit(X_train, X_train,  # X_train twice!
                epochs=10000,
                batch_size=64,
                shuffle=True,
                validation_split=.2,
                verbose = True,
                callbacks=[stop, tensorboard])

Epoch 1/10000
  1/838 [..............................] - ETA: 0s - loss: 0.6953WARNING:tensorflow:From /opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
838/838 [==============================] - 2s 2ms/step - loss: 0.3158 - val_loss: 0.2576
Epoch 2/10000
838/838 [==============================] - 1s 2ms/step - loss: 0.2437 - val_loss: 0.2343
Epoch 3/10000
838/838 [==============================] - 1s 2ms/step - loss: 0.2306 - val_loss: 0.2279
Epoch 4/10000
838/838 [==============================] - 1s 2ms/step - loss: 0.2263 - val_loss: 0.2248
Epoch 5/10000
838/838 [==============================] - 1s 2ms/step - loss: 0.2244 - val_loss: 0.2234
Epoch 6/10000
838/838 [==============================] - 1s 2ms/step - loss: 0.2235 - val_loss: 0.2227
Epoch 7/10000
838/83

In [13]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import os
import datetime
%tensorboard --logdir ./logs

Reusing TensorBoard on port 6006 (pid 2275), started 1:09:06 ago. (Use '!kill 2275' to kill it.)

In [14]:
# encode and decode some digits

decoded_imgs = autoencoder(X_train)

In [17]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

# Create Model 

autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

AttributeError: 'numpy.ndarray' object has no attribute 'map'

In [21]:
X_train.shape, y_train.shape

((67000, 784), (67000, 1))

In [23]:
import os
import datetime

stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=2)

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

autoencoder.fit(X_train, y_train,
                epochs=10000,
                batch_size=64,
                shuffle=True,
                validation_split=.33,
                verbose = False,
                callbacks=[stop, tensorboard])

ValueError: in user code:

    /opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/anaconda3/envs/U4-S1-NLP

### Stretch Goals
- Implement Bayesian Hyper-parameter Optimization
- Select a new dataset and apply a neural network to it.
- Use a cloud base experiment tracking framework such as weights and biases
- Research potential architecture ideas for this problem. Try Lenet-10 for example. 